<a href="https://colab.research.google.com/github/Jsgithubchannel/deep_learning_practice/blob/main/situation_model_changed_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 코랩 환경 설정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 46.9 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=f3dbb7343082a8caa2f28e34d368ff1d2778da226337a51d3792d01b2e932770
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 7.2 MB/s 
     |████████████████████████████████| 3.1 MB 9.9 MB/s 
     |████████████████████████████████| 596 kB 38.8 MB/s 
     |████████████████████████████████| 3.3 MB 66.4 MB/s 
     |████████████████████████████████| 895 kB 69.7 MB/s 
     |████████████████████████████████| 61 kB 661 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13


In [2]:
#Kobert 파일 로드 from github
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-35yoxwpx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-35yoxwpx
     |████████████████████████████████| 4.8 MB 8.7 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13140 sha256=51d27d2a8df3e00ab16da22071023c88ce1c8d4a562d454d592db07baed4c0db
  Stored in directory: /tmp/pip-ephem-wheel-cache-w5a4grmd/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
# 환경 설정 (필요한 모듈 import)
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
#GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 2. 데이터셋 전처리
import pandas as pd
chat_data = pd.read_excel('/content/drive/MyDrive/capstone_design/reduced_situation_dataset.xlsx')

In [7]:
print(len(chat_data)) # 데이터 개수
chat_data.sample(n=10) # 랜덤 데이터 10개 출력

49660


,situation,sentence1,sentence2,sentence3
25121,"연애, 결혼, 출산",이제 슬슬 아기가 낳을때가 된것같아.,이제 슬슬 아기용품을 준비해야될것같아.,아이가 아빠를 닮았으면 좋겠어.
14996,대인관계,친구가 특별한 용건이 없어도 매일 전화를 걸어서 시시콜콜한 이야기를 하는 게 귀찮고...,나와는 다르게 서로의 일상을 전부 나누고 이야기할 수 있는 편안한 시간이라고 생각할...,상처받지 않게 조심스럽게 이야기를 해야 할 것 같아. 매일같이 통화하는 건 이제 좀...
9223,"건강, 죽음",급격히 내가 한심한 느낌이 들어.,건강하지 못해서 짐이 되는 느낌이 들거든.,내가 할 수 있는 일을 조금씩 해보려고.
47882,학업 및 진로,엄마한테 내가 하고 싶은 것을 얘기했는데 별 반응이 없으셔.,엄마 반응이 별로 좋지 않았어.,아직 내가 얼마만큼 그걸 하고 싶어 하는지 엄마는 몰라. 앞으로 꾸준히 내 꿈을 이...
10096,"건강, 죽음",친구는 죽음이 두렵대.,누구나 겪어야 하는 일이잖아. 인정하면 그때부터 마음이 편해지지.,그것도 누구에게나 똑같지. 마찬가지로 인정하면 두렵지않아.
16288,대인관계,새로 입사한 회사 사람들은 가자 무리가 있어서 친해지기 어려워고립된 기분이야.,회사 사람들이 새로운 사람을 챙겨주면 좋겠어.,동료들에게 다가가 말을 걸어볼 용기가 필요해.
5843,건강,요즘 골프 연습을 많이 하네!,연습 많이 하면 허리도 날씬해지고 좋은 것 같아.,즐거운 운동을 위해 현재 건강을 최선으로 관리가 필요해.
3985,건강,얼마 전에 감기에 걸렸는데 가족들한테 옮길까 봐 많이 신경 쓰였어.,나이가 드니까 감기에 걸리면 다른 병으로 이어질까 봐 무섭더라고.,다행히 감기는 금방 나았는데 건강관리를 못 했던 거 같아서 앞으로 조심해야겠어.
3925,건강,나이 보다 건강하다고 생각했는데 삼십 대 이전으로 건강을 회복할 수 없을 거라는 의...,나이가 들어가는 자연스러운 현상이지만 그래서 서글픈 생각이 들어,늙는다는 걸 받아들이고 몸과 마음이 모두 건강할 수 있도록 노력해야겠어.
27543,"연애, 결혼, 출산",나 이번에 결혼해.,그런데 만난 지 두 달 만에 속도위반이라,그래도 부모님께 너무 부끄러워.


In [8]:
situations = ['가족관계','학업 및 진로','학교폭력/따돌림','대인관계(부부, 자녀)','대인관계','연애, 결혼, 출산', '진로, 취업, 직장', '재정, 은퇴, 노후준비', '건강, 죽음', '건강', '직장, 업무 스트레스', '재정'] # 감정 대분류
for i in range(len(situations)):
  chat_data.loc[(chat_data['situation'] == situations[i]), 'situation'] = i

In [9]:
data_list = []
for label, sentence1, sentence2, sentence3 in zip(chat_data['situation'], chat_data['sentence1'], chat_data['sentence2'], chat_data['sentence3']):
    data = []
    sentence = sentence1+' '+sentence2+' '+sentence3
    data.append(sentence)
    data.append(str(label))

    data_list.append(data)

In [10]:
data_list[49000]

['공부가 내 길이 아니라 공부를 안하는 것 뿐이야. 공부가 내 길이 아니라 가수가 내 길이라서 안하는거 뿐이야. 난 노래 연습을 할거야 그거면 돼.',
 '1']

In [11]:
# 3. Train data & Test data
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [12]:
# 4. KoBERT 입력 데이터로 만들기

# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [27]:
# Setting parameters
max_len = 200
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
print(len(dataset_train))
print(len(dataset_test))

39728
9932


In [15]:
#토큰화
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

#BERT 모델, Vocabulary 불러오기
from kobert.pytorch_kobert import get_kobert_model
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file) 

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-81llehv1/kobert-tokenizer_3fc070191104459eb573bece66b5734b
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-81llehv1/kobert-tokenizer_3fc070191104459eb573bece66b5734b
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=ceff6191b2e861e6cc0254d769c4516c75a0649da1e5a6b14d875c78b2a6c69d
  Stored in directory: /tmp/pip-ephem-wheel-cache-qtycvczd/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

In [16]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok,vocab,  max_len, True, False)

In [17]:
data_train[0]

(array([   2, 4162, 6356, 5468, 7234, 1846, 3111, 5398, 5859, 3135, 7828,
        4627, 5330, 1567, 3194, 5940, 6022, 2752, 5931, 3191, 3574, 6585,
        1852, 5760,  517, 5617, 7096, 3128,  517, 5499, 7350, 5663, 5782,
          54,   54, 2251, 7832, 3191, 1958, 6150, 2307, 5920, 6113, 6113,
        7784, 4966, 1562, 5330, 3105, 5850,  517, 5617, 6368, 3059, 6553,
        5100, 5671, 5702,   54, 3192, 5760, 4799, 7659, 6116,  517, 7375,
        5357, 6553, 1610, 2437, 7088,  775, 2251, 7788, 4435, 3135, 7811,
        4077, 6116, 2086, 6291, 5859, 1189, 4156, 7836, 5377, 6844,   54,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [18]:
# torch 형식의 dataset
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
# 5. KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=12, ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [28]:
# KoBERT 모델 학습시키기
path_list = ['/content/drive/MyDrive/capstone_design/model_epoch18.pth', '/content/drive/MyDrive/capstone_design/model_epoch19.pth', '/content/drive/MyDrive/capstone_design/model_epoch20.pth', '/content/drive/MyDrive/capstone_design/model_epoch21.pth', '/content/drive/MyDrive/capstone_design/model_epoch22.pth']
path_idx = 0

for e in range(epoch, epoch + num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    state = {
        'epoch': num_epochs,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'loss': loss,
    }
    path = path_list[path_idx]
    torch.save(state,path)
    path_idx += 1

epoch = epoch + num_epochs

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/2483 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.0007786502246744931 train acc 1.0
epoch 3 batch id 201 loss 0.0009746834984980524 train acc 0.9916044776119403
epoch 3 batch id 401 loss 0.0005332333385013044 train acc 0.9883104738154613
epoch 3 batch id 601 loss 0.03883521631360054 train acc 0.9733777038269551
epoch 3 batch id 801 loss 0.12283642590045929 train acc 0.9624687890137328
epoch 3 batch id 1001 loss 0.007027919869869947 train acc 0.9556068931068931
epoch 3 batch id 1201 loss 0.016434406861662865 train acc 0.9520191507077436
epoch 3 batch id 1401 loss 0.003692707745358348 train acc 0.9490096359743041
epoch 3 batch id 1601 loss 0.030817154794931412 train acc 0.9474156777014366
epoch 3 batch id 1801 loss 0.13627101480960846 train acc 0.9459328151027208
epoch 3 batch id 2001 loss 0.020090362057089806 train acc 0.9459332833583208
epoch 3 batch id 2201 loss 0.4875829815864563 train acc 0.9457916855974557
epoch 3 batch id 2401 loss 0.0070501528680324554 train acc 0.9460381091211995
epoch 3 train acc 0.94

  0%|          | 0/621 [00:00<?, ?it/s]

epoch 3 test acc 0.6917270531400966


...

epoch 6 test acc 0.6855542136339238


  0%|          | 0/2483 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.19645845890045166 train acc 0.9375
epoch 7 batch id 201 loss 0.008771345019340515 train acc 0.9300373134328358
epoch 7 batch id 401 loss 0.2707147002220154 train acc 0.9351620947630923
epoch 7 batch id 601 loss 0.06639882922172546 train acc 0.9389559068219634
epoch 7 batch id 801 loss 0.3159770369529724 train acc 0.9420255930087391
epoch 7 batch id 1001 loss 0.007387724705040455 train acc 0.9443681318681318
epoch 7 batch id 1201 loss 0.004717959091067314 train acc 0.9465549542048293
epoch 7 batch id 1401 loss 0.0036890774499624968 train acc 0.9488758029978587
epoch 7 batch id 1601 loss 0.061359092593193054 train acc 0.9499141161773891
epoch 7 batch id 1801 loss 0.049496475607156754 train acc 0.9516588006662965
epoch 7 batch id 2001 loss 0.12572240829467773 train acc 0.9538043478260869
epoch 7 batch id 2201 loss 0.0015208038967102766 train acc 0.9556451612903226
epoch 7 batch id 2401 loss 0.0012501770397648215 train acc 0.9568148688046647
epoch 7 train acc 0.95

  0%|          | 0/621 [00:00<?, ?it/s]

epoch 7 test acc 0.6992418142780462


In [ ]:
# 저장하기
state = {
        'epoch': num_epochs,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'loss': loss,
}
path ="/content/drive/MyDrive/capstone_design/model_epoch23.pth"
torch.save(state,path)

In [29]:
# 불러오기
checkpoint = torch.load("/content/drive/MyDrive/capstone_design/model_epoch22.pth")
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# model.train()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("가족관계")
            elif np.argmax(logits) == 1:
                test_eval.append("학업 및 진로")
            elif np.argmax(logits) == 2:
                test_eval.append("학교폭력/따돌림")
            elif np.argmax(logits) == 3:
                test_eval.append("대인관계(부부, 자녀)")
            elif np.argmax(logits) == 4:
                test_eval.append("대인관계")
            elif np.argmax(logits) == 5:
                test_eval.append("연애, 결혼, 출산")
            elif np.argmax(logits) == 6:
                test_eval.append("진로, 취업, 직장")
            elif np.argmax(logits) == 7:
                test_eval.append("재정, 은퇴, 노후준비")
            elif np.argmax(logits) == 8:
                test_eval.append("건강, 죽음")
            elif np.argmax(logits) == 9:
                test_eval.append("건강")
            elif np.argmax(logits) == 10:
                test_eval.append("직장, 업무 스트레스")
            elif np.argmax(logits) == 11:
                test_eval.append("재정")

            print(logits)
        print(">> 입력하신 내용에서 " + test_eval[0] + " 상황입니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료

end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 공부가 내 길이 아니라 공부를 안하는 것 뿐이야. 공부가 내 길이 아니라 가수가 내 길이라서 안하는거 뿐이야. 난 노래 연습을 할거야 그거면 돼.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[-0.08626291  7.5850387   1.0644163   0.12527494  0.2832121  -2.4605815
  0.5631924  -0.43216223 -1.5626361   0.08037304 -2.4776974  -0.572239  ]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 공부는 힘들어
[-0.8184731   7.1750045   1.5238721   0.7509221   0.39652944 -2.6210127
 -0.0325031   0.01507865 -1.4918983   0.36574253 -2.4716053  -0.8044045 ]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 돈버는 것도 힘들어
[-2.5421252  -0.7501472  -1.6436696  -0.95584273  0.85087025 -2.2679324
  1.2098463   2.2372916  -1.0911473  -0.9288866  -0.34687385  5.5733123 ]
>> 입력하신 내용에서 재정 상황입니다.


하고싶은 말을 입력해주세요 : 삶의 의미는 뭘까?
[-0.1750377   2.8037302   0.6891271  -1.555724    0.370161   -1.8484235
  1.3590399   0.45635992  2.256129   -1.1854153  -2.4654038  -0.20953509]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 요즘 몸이 안좋아서 필라테스 다녀왔어
[ 1.230831   -1.0924592  -1.3921441  -2.224237    1.3496826  -0.09393229
 -2.1893322  -1.9079479   6.3059173   1.1123224  -2.4035513  -0.3818108 ]
>> 입력하신 내용에서 건강, 죽음 상황입니다.


하고싶은 말

In [ ]:
#캐시 삭제
import gc
gc.collect()
torch.cuda.empty_cache()